In [2]:
library(haven)
library(tidyverse)

# 1. Clean up BRFSS datasets and select exploratory variables

I downloaded the SMART: BRFSS City and County Data and Documentation from the CDC website:
`https://www.cdc.gov/brfss/smart/Smart_data.htm`

However, not all years had available data - these are the years for which I could download the dataset:

In [3]:
survey_files = list.files('../../Data/BRFSS_dataset/')
survey_files

[1] "MMSA2003.xpt" "MMSA2004.xpt" "MMSA2005.xpt" "MMSA2006.xpt" "MMSA2008.xpt"
 [6] "MMSA2013.xpt" "MMSA2014.xpt" "MMSA2015.xpt" "MMSA2016.xpt" "MMSA2017.xpt"
[11] "MMSA2018.xpt" "MMSA2019.xpt"

In [4]:
survey_repo = list()
for (file in survey_files) {
    year = substr(file, 5, 8)
    tbl = read_xpt(paste0('../../Data/BRFSS_dataset/', file))
    print(dim(tbl))
    survey_repo[[year]] = tbl
}

[1] 129779    296
[1] 163168    288
[1] 201539    317
[1] 197744    277
[1] 241150    264
[1] 256271    154
[1] 243476    147
[1] 229015    153
[1] 249011    146
[1] 230875    177
[1] 221587    145
[1] 210771    179


We can see that there is a large variance in the number of columns per year - we'll only select a few interesting variables for this exploratory analysis.

In [5]:
head(survey_repo[['2003']])

_MMSANAM,_LMSANAM,FMONTH,IDATE,IMONTH,IDAY,IYEAR,INTVID,DISPCODE,SEQNO,...,ADJMMSA,_MMSAWT,_LMMSA,SEX_LM,AGE_LM,RACE_LM,RACE_LMF,AGE_LMF,ADJLMMSA,_LMMSAWT
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Albuquerque, NM",,1,01222003,01,22,2003,164,120,2003000439,...,4.874913,320.4042,NA,NA,NA,NA,NA,NA,NA,NA
"Albuquerque, NM",,2,02112003,02,11,2003,188,110,2003000702,...,4.874913,640.8084,NA,NA,NA,NA,NA,NA,NA,NA
"Albuquerque, NM",,1,01212003,01,21,2003,129,110,2003006985,...,4.874913,482.4036,NA,NA,NA,NA,NA,NA,NA,NA
"Albuquerque, NM",,3,03052003,03,05,2003,188,110,2003008064,...,4.874913,1286.5426,NA,NA,NA,NA,NA,NA,NA,NA
"Albuquerque, NM",,4,04122003,04,12,2003,129,110,2003015321,...,4.874913,964.9070,NA,NA,NA,NA,NA,NA,NA,NA
"Albuquerque, NM",,4,04082003,04,08,2003,129,110,2003015383,...,4.874913,964.9070,NA,NA,NA,NA,NA,NA,NA,NA


The columns that start with _ have been calculated by the analysts at CDC and formatted for us to use - I'll use these columns. Namely, I'll match columns that start with:

- MMSANAM - Name of the MSA
- _AGEG5YR
- _SEXG = this variable actually is pretty irregular over the years - not used at this stage
- _TOTINDA : variable for exercise
- _FLUSHOT/_FLSHOT : vaccination
- _RFBING : binge drinking
- _RFDRHV : heavy drinking
- _SMOKER : level of smoking
- _RFSMOK : current smokers

In [5]:
match_cols = c('MMSANAM', '_AGEG5YR', '_INCOMG', '_TOTINDA', '_FL', '_RFBING', '_RFDRHV', '_SMOKER', '_RFSMOK')
survey_clean = list()
for (file in survey_files) {
    year = substr(file, 5, 8)
    matched_cols = sapply(match_cols, function(x) {which(grepl(x , names(survey_repo[[year]])))})
    cleaned_table = cbind(survey_repo[[year]][, c('_MMSA')], survey_repo[[year]][, matched_cols])
    print(dim(cleaned_table))
    colnames(cleaned_table) = c('mmsa', 'mmsa_name', 'age', 'income', 'exercise', 'flushot',
        'binging', 'heavy_drink', 'smoker', 'smoking_now')
    cleaned_table$year = as.numeric(year)
    survey_clean[[year]] = cleaned_table
    # I tried combining the table, but ran into memory issues (personal computer) - opting for saving each separately
    # need to write .tsv, since MMSA names have commas in them
#     write.table(cleaned_table, file = paste0('../../Table/BRFSS/cleaned_10cols_', year, '.tsv'),
#         row.names = F, quote = F, sep = '\t')
}

[1] 129779     10
[1] 163168     10
[1] 201539     10
[1] 197744     10
[1] 241150     10
[1] 256271     10
[1] 243476     10
[1] 229015     10
[1] 249011     10
[1] 230875     10
[1] 221587     10
[1] 210771     10


In [6]:
# let's save some memory:
rm(survey_repo)
total_tbl = do.call('rbind', survey_clean)
rm(survey_clean)

In [7]:
dim(total_tbl)
head(total_tbl)

[1] 2574386      11

,mmsa,mmsa_name,age,income,exercise,flushot,binging,heavy_drink,smoker,smoking_now,year
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2003.1,10740,"Albuquerque, NM",1,3,1,NA,2,1,1,2,2003
2003.2,10740,"Albuquerque, NM",1,2,1,NA,2,1,3,1,2003
2003.3,10740,"Albuquerque, NM",1,4,1,NA,2,1,2,2,2003
2003.4,10740,"Albuquerque, NM",1,5,1,NA,1,1,1,2,2003
2003.5,10740,"Albuquerque, NM",1,3,1,NA,1,1,4,1,2003
2003.6,10740,"Albuquerque, NM",1,2,1,NA,1,1,4,1,2003


In [11]:
# some basic stats on each column - we have 3 range variables and 5 binary variables
length(unique(total_tbl$mmsa)) # data available from 265 MSA's
unique(total_tbl$age) # range (1 to 14)
unique(total_tbl$income) # range (1 to 5)
unique(total_tbl$exercise) # binary (1 or 2)
unique(total_tbl$flushot) # binary (1 or 2)
unique(total_tbl$binging) # binary (1 or 2)
unique(total_tbl$heavy_drink) # binary (1 or 2)
unique(total_tbl$smoker) # range (1 to 4)
unique(total_tbl$smoking_now) # binary (1 or 2)
unique(total_tbl$year) # years 2003 to 2019, but some gaps in between, including 4 years between 2008 and 2013

[1] 265

[1]  1  2  3  4  5 14  6  7  8  9 10 11 12 13

[1] 3 2 4 5 9 1

[1] 1 2 9

[1] NA  9  2  1

[1] 2 1 9

[1] 1 2 9

[1] 1 3 2 4 9

[1] 2 1 9

[1] 2003 2004 2005 2006 2008 2013 2014 2015 2016 2017 2018 2019

# 2. Matching MSAs to state, and combining another dataset with MSA available (HPI data)

In order to combine this dataset with other datasets that have either MSA-level or state-level data, I need to do some additional processing. First, I'm going to convert each MSA to state, but with the rule thumb that in case of multi-state MSAs (such as `Chicago-Naperville-Joliet, IL-IN-WI`), the first state will be chosen (e.g. `IL`), since it is the state with the largest representation in most cases.

In [8]:
z = 'Albuquerque, NM'
strsplit(z, ', ')[[1]][2]
z = 'Boston-Quincy, MA Metropolitan Division'
strsplit(strsplit(z, ', ')[[1]][2], ' ')[[1]][1]
z = 'Washington-Arlington-Alexandria, DC-VA-MD-WV Metropolitan Division'
strsplit(strsplit(strsplit(z, ', ')[[1]][2], ' ')[[1]][1], '-')[[1]][1]

[1] "NM"

[1] "MA"

[1] "DC"

In [9]:
# Let's extract the state variable (two-letter) from each row:
extract_state = function(name) {
    return(
        strsplit(
            strsplit(
                strsplit(name, ', ')[[1]][2],
            ' ')[[1]][1],
        '-')[[1]][1]
    )
}
total_tbl$state = sapply(total_tbl$mmsa_name, function(x) {extract_state(x)})

In [10]:
unique(total_tbl$state)
length(unique(total_tbl$state))

[1] "NM" "AK" "NC" "GA" "MD" "LA" "AL" "ID" "MA" "WA" "CT" "VT" "NJ" "WY" "WV"
[16] "SC" "IL" "OH" "NH" "TX" "CO" "IA" "MI" "DE" "ND" "AR" "HI" "IN" "MS" "MO"
[31] "NV" "NE" "CA" "KY" "TN" "FL" "WI" "MN" "NY" "UT" "OK" "PA" "AZ" "ME" "OR"
[46] "RI" "SD" "VA" "KS" "DC" "MT" "PR"

[1] 52

Meanwhile, we can also attach the Housing Price Index (HPI) values to each entry - this will supplement the income column by providing additional insight into the state of the regional economy:
The MSA-level HPI from 1991 to 2021 is available here:

`https://www.fhfa.gov/DataTools/Downloads/Pages/Public-Use-Databases.aspx`

It will also be possible to go into even more detail by taking zip-code specific tax returns, but that is outside the scope of this analysis:

`https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi`


### Attach HPI values to each entry using MSA code and year
I'm going to use only the first month - we can also make this more finely detailed later by looking at the month of the call and the corresponding entry

In [12]:
hpi_tbl = read.csv('../../Data/HPI_master.csv', stringsAsFactors = F)
hpi_sub_tbl = hpi_tbl %>% filter(level == 'MSA') %>% distinct(place_id, yr, .keep_all = T)
head(hpi_sub_tbl)

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,2,108.26,NA
2,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,1,101.17,NA
3,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1988,1,82.03,NA
4,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1989,1,87.87,NA
5,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1990,1,81.25,NA
6,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1991,1,83.35,NA


In [13]:
id = 48300
head(hpi_sub_tbl %>% filter(place_id == id))
head(total_tbl %>% filter(mmsa == id))

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,traditional,all-transactions,quarterly,MSA,"Wenatchee, WA",48300,1978,3,42.53,NA
2,traditional,all-transactions,quarterly,MSA,"Wenatchee, WA",48300,1979,1,44.45,NA
3,traditional,all-transactions,quarterly,MSA,"Wenatchee, WA",48300,1980,1,45.67,NA
4,traditional,all-transactions,quarterly,MSA,"Wenatchee, WA",48300,1981,2,53.56,NA
5,traditional,all-transactions,quarterly,MSA,"Wenatchee, WA",48300,1982,1,25.17,NA
6,traditional,all-transactions,quarterly,MSA,"Wenatchee, WA",48300,1983,1,48.84,NA


,mmsa,mmsa_name,age,income,exercise,flushot,binging,heavy_drink,smoker,smoking_now,year,state
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2003.124972,48300,"Wenatchee, WA",1,9,1,NA,2,1,4,1,2003,WA
2003.124973,48300,"Wenatchee, WA",1,5,2,NA,1,1,4,1,2003,WA
2003.124974,48300,"Wenatchee, WA",1,1,1,NA,1,1,1,2,2003,WA
2003.124975,48300,"Wenatchee, WA",1,3,1,NA,2,2,1,2,2003,WA
2003.124976,48300,"Wenatchee, WA",1,1,1,NA,2,1,1,2,2003,WA
2003.124977,48300,"Wenatchee, WA",1,9,2,NA,1,1,4,1,2003,WA


In [14]:
# Let's do a left_join() with index_nsa:
hpi_join_tbl = hpi_sub_tbl %>% select(c('place_id', 'yr', 'index_nsa')) %>% rename(
    mmsa = place_id, year = yr, hpi = index_nsa)
hpi_join_tbl$mmsa = as.numeric(hpi_join_tbl$mmsa)
head(hpi_join_tbl)
total_tbl_hpi = total_tbl %>% left_join(hpi_join_tbl, by = c('mmsa', 'year'))

,mmsa,year,hpi
,<dbl>,<int>,<dbl>
1,10180,1986,108.26
2,10180,1987,101.17
3,10180,1988,82.03
4,10180,1989,87.87
5,10180,1990,81.25
6,10180,1991,83.35


In [16]:
head(total_tbl_hpi)



,mmsa,mmsa_name,age,income,exercise,flushot,binging,heavy_drink,smoker,smoking_now,year,state,hpi
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,10740,"Albuquerque, NM",1,3,1,NA,2,1,1,2,2003,NM,120.21
2,10740,"Albuquerque, NM",1,2,1,NA,2,1,3,1,2003,NM,120.21
3,10740,"Albuquerque, NM",1,4,1,NA,2,1,2,2,2003,NM,120.21
4,10740,"Albuquerque, NM",1,5,1,NA,1,1,1,2,2003,NM,120.21
5,10740,"Albuquerque, NM",1,3,1,NA,1,1,4,1,2003,NM,120.21
6,10740,"Albuquerque, NM",1,2,1,NA,1,1,4,1,2003,NM,120.21


In [15]:
# some entries are missing hpi, but that is okay - we'll save the table
sum(is.na(total_tbl_hpi$hpi))

write.table((total_tbl_hpi %>% select(-c('mmsa_name'))), 
    file = '../../Table/BRFSS/BRFSS_cleaned_master.tsv', quote = F, row.names = F, sep = '\t')

[1] 302957

In [22]:
# I'm also going to save the MSA names into a separate file, since it's a bit clunky:
mmsa_tbl = total_tbl_hpi %>% distinct(mmsa, .keep_all = T) %>% 
    select(c('mmsa', 'state', 'mmsa_name')) %>% arrange(state, mmsa_name)
write.table(mmsa_tbl, file = '../../Table/US_MMSA.tsv', quote = F, row.names = F, sep = '\t')

# 3. Clean up MSU state policy dataset and select exploratory variables

MSU has a dataset on correlates of state policy (CSPP)
Downloaded from: `http://ippsr.msu.edu/public-policy/correlates-state-policy`

There are two data files - the full csv dataset and the policy metadata file:

In [24]:
# main file
msu_tbl = read.csv('../../Data/MSU_CSPP/cspp_june_2021.csv', header = T, stringsAsFactors = F)
dim(msu_tbl)

# metadata file
codebook = read.csv('../../Data/MSU_CSPP/codebook.csv', header = T, stringsAsFactors = F) %>% drop_na(category)
sum(codebook$category == 'demographics')
sum(codebook$category == 'education')
sum(codebook$category == 'healthcare')
sum(codebook$category == 'welfare')
sum(codebook$category == 'drug-alcohol')

[1] 6281 2142

[1] 45

[1] 93

[1] 220

[1] 24

[1] 122

In [25]:
unique(codebook$category)

[1] "demographics"     "economic-fiscal"  "environment"      "government"      
 [5] "elections"        "policy-ideology"  "criminal justice" "education"       
 [9] "healthcare"       "welfare"          "rights"           "drug-alcohol"    
[13] "gun control"      "labor"            "transportation"   "misc. regulation"

In [26]:
# This is a pretty extensive dataset with > 2000 correlates. Let's narrow it down a bit
# First, we're only doing correlation, so all years before 2003 are not used:
cleaned_tbl = msu_tbl %>% filter(year >= 2003) %>% distinct(year, st, .keep_all = T)

# Let's narrow down to demographics and policies on education, healthcare, welfare and drug-alcohol:
# we need to import the metadata:
# categories = c('demographics', 'education', 'healthcare', 'welfare', 'drug-alcohol')
# codebook_subset = codebook %>% filter(category %in% categories)

# cleaned_tbl = cleaned_tbl[, c('year', 'st', 'state', codebook_subset$variable)]
# dim(cleaned_tbl)

In [27]:
tail(cleaned_tbl)

,year,st,stateno,state,state_fips,state_icpsr,popdensity,popfemale,pctpopfemale,popmale,...,med_spending_own,poptotal,taxes,taxrevcorporate,total_debt_outstanding,total_expenditure,total_revenue,popnohealthins,popprivhealthins,popgovhealthins
,<int>,<chr>,<dbl>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,...,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
862,2019,VT,45,Vermont,50,6,NA,NA,NA,NA,...,NA,623989,3470585,149832,NA,NA,NA,28,425,259
863,2019,VA,46,Virginia,51,40,NA,NA,NA,NA,...,NA,8535519,27076451,1247128,NA,NA,NA,658,6196,2551
864,2019,WA,47,Washington,53,73,NA,NA,NA,NA,...,NA,7614893,27992437,NA,NA,NA,NA,496,5327,2641
865,2019,WV,48,West Virginia,54,56,NA,NA,NA,NA,...,NA,1792147,5816816,198799,NA,NA,NA,118,1103,838
866,2019,WI,49,Wisconsin,55,25,NA,NA,NA,NA,...,NA,5822434,19930137,1343532,NA,NA,NA,329,4275,1907
867,2019,WY,50,Wyoming,56,68,NA,NA,NA,NA,...,NA,578759,2110704,NA,NA,NA,NA,70,407,166


In [28]:
# Let's have some focus groups - first that comes to mind is taxation:
# Let's narrow it down further to a few categories
categories = c('environment', 'healthcare', 'drug-alcohol', 'misc. regulation')
codebook_subset = codebook %>% filter(category %in% categories)
tax_columns = codebook_subset[which(sapply(codebook_subset$long_desc, function(x) {grepl('tax', x)})), 'variable']
tax_columns

[1] "fgastax1"                      "w_environment_solar_taxcredit"
 [3] "fgastax1"                      "cigtax"                       
 [5] "beer_tax"                      "beer_tax_rank"                
 [7] "spirit_tax"                    "spirit_tax_rank"              
 [9] "wine_tax"                      "wine_tax_rank"                
[11] "cbeertex"                      "cbeerp"                       
[13] "cbeert"                        "cbeertav"                     
[15] "cwinep"                        "cwinet"                       
[17] "cwinetav"                      "cwinetex"                     
[19] "cspirp"                        "cspirt"                       
[21] "cspirtav"                      "cspirtex"                     
[23] "icigtaxraw"                    "icigtax"                      
[25] "iindex"                        "cig_taxes_binary"             
[27] "z_cigarette_taxes"

In [29]:
# We can also look at % of income spending:
spend_columns = codebook[which(sapply(codebook$long_desc, function(x) {grepl('% of income', x)})), 'variable']
spend_columns

[1] "aairpi"    "aasstpi"   "aasstpia"  "acorrpi"   "adebtpi"   "adebtpia" 
 [7] "aedpi"     "aerpi"     "aesapi"    "afcgapi"   "agpbpi"    "agrantspi"
[13] "ahcdpi"    "ahealpi"   "ahwypi"    "aincloc3"  "aincloc4"  "aincstat3"
[19] "aincstat4" "ainctot2"  "ainctot3"  "ainctot4"  "aintpi"    "alibpi"   
[25] "anecpi"    "anrpi"     "aoitpi"    "aparkpi"   "apkgpi"    "apolpi"   
[31] "apwpi"     "asanpi"    "atotpi"    "atotpib"   "atotpic"   "aunempi"  
[37] "autilpi"   "awtpi"

In [30]:
# Let's save these two files:
cleaned_tbl_subset = cleaned_tbl[, c('year', 'st', 'state', tax_columns)]
# remove columns that are all NA
cleaned_tbl_subset = cleaned_tbl_subset[, 
    sapply(names(cleaned_tbl_subset), function(x) {sum(is.na(cleaned_tbl_subset[, x])) < nrow(cleaned_tbl_subset)})]
write.table(cleaned_tbl_subset, 
    file = '../../Table/state_policy_tax.tsv', quote = F, row.names = F, sep = '\t')

cleaned_tbl_subset = cleaned_tbl[, c('year', 'st', 'state', spend_columns)]
# remove columns that are all NA
cleaned_tbl_subset = cleaned_tbl_subset[, 
    sapply(names(cleaned_tbl_subset), function(x) {sum(is.na(cleaned_tbl_subset[, x])) < nrow(cleaned_tbl_subset)})]
write.table(cleaned_tbl_subset, 
    file = '../../Table/state_policy_spending.tsv', quote = F, row.names = F, sep = '\t')